In [2]:
# библиотеки общего назначения
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# для работы с файлами
import glob

# для проверки суррогатных моделей
import sklearn.metrics
from sklearn.model_selection import train_test_split, GridSearchCV

# для создания моих суррготаных моделей
import MY_SOLVERS as mysol

# для работы с моим интегратором
import my3D_surface_integrator as sint

# для работы с моим крылом
from MY_WINGS import wing1

# для комплексной проверки качества суррогатной модели
import quality_PCA_model as qpca # купец-а

# для сохранения и загрузки суррогатных моделей
import joblib

# для отслеживания прогресса
from tqdm import tqdm

# Чтение данных

In [3]:
regimes = (pd.read_csv("./data/regimes.csv")).drop(['Num'],axis=1)
M_is = pd.DataFrame(columns=np.arange(regimes.shape[0]))
Cp = pd.DataFrame(columns=np.arange(regimes.shape[0]))
Cf = pd.DataFrame(columns=np.arange(regimes.shape[0]))

files = glob.glob("data_Cf/*_*_*.csv")
files.sort()
XYZ_coords = []
XYZ_speeds = []
for i,path in tqdm(enumerate(files)):
    M_is[i] = pd.read_csv(path)['M_is']
    Cp[i] = pd.read_csv(path)['Cp']
    Cf[i] = pd.read_csv(path)['Cf']
    XYZ_coords.append(pd.read_csv(path)[['Points:0','Points:1','Points:2']])
    XYZ_speeds.append(pd.read_csv(path)[['Velocity:0','Velocity:1','Velocity:2']])

S_REF = 0.15

64it [00:41,  1.54it/s]


In [4]:
regimes

,M,alpha
0,0.94648,-3.71875
1,0.83867,-2.40625
2,0.81289,5.84375
3,0.84102,3.59375
4,0.89727,2.65625
...,...,...
59,0.85273,4.34375
60,0.82930,0.78125
61,0.85039,-3.34375
62,0.89258,4.90625


In [5]:
Cf

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.003431,0.003564,0.000906,0.001539,0.001863,0.002300,0.000977,0.001896,0.000833,0.002586,...,0.001009,0.003100,0.003775,0.001057,0.002805,0.001360,0.002433,0.003835,0.001296,0.002962
1,0.003315,0.003431,0.000815,0.001351,0.001690,0.002122,0.000879,0.001712,0.000788,0.002411,...,0.000913,0.002947,0.003651,0.000921,0.002655,0.001183,0.002256,0.003708,0.001135,0.002827
2,0.003311,0.003417,0.000774,0.001196,0.001551,0.001993,0.000822,0.001564,0.000793,0.002297,...,0.000853,0.002892,0.003653,0.000826,0.002583,0.001040,0.002133,0.003710,0.001006,0.002782
3,0.003333,0.003428,0.000796,0.001058,0.001423,0.001876,0.000819,0.001427,0.000865,0.002199,...,0.000842,0.002860,0.003683,0.000780,0.002532,0.000920,0.002024,0.003739,0.000904,0.002761
4,0.003203,0.003276,0.000894,0.000922,0.001266,0.001697,0.000884,0.001258,0.001006,0.002018,...,0.000894,0.002696,0.003543,0.000800,0.002374,0.000827,0.001844,0.003597,0.000836,0.002615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86956,0.002736,0.003705,0.010728,0.010378,0.009360,0.007021,0.010120,0.009577,0.010677,0.005100,...,0.013499,0.002651,0.002514,0.010896,0.002566,0.010169,0.006679,0.002766,0.010131,0.002838
86957,0.002819,0.003728,0.010792,0.010398,0.009349,0.007013,0.010114,0.009537,0.010722,0.005032,...,0.013603,0.002660,0.002553,0.010922,0.002573,0.010218,0.006638,0.002812,0.010147,0.002810
86958,0.002900,0.003748,0.010851,0.010412,0.009338,0.007006,0.010105,0.009499,0.010761,0.004967,...,0.013682,0.002668,0.002590,0.010943,0.002578,0.010263,0.006597,0.002856,0.010155,0.002779
86959,0.002978,0.003769,0.010905,0.010423,0.009327,0.006999,0.010092,0.009462,0.010792,0.004904,...,0.013739,0.002676,0.002627,0.010960,0.002585,0.010305,0.006554,0.002900,0.010160,0.002746


In [6]:
XYZ_coords[0]

,Points:0,Points:1,Points:2
0,4.531483,-0.045957,0.094869
1,4.531380,-0.045805,0.094880
2,4.531278,-0.045653,0.094890
3,4.531182,-0.045496,0.094901
4,4.531093,-0.045335,0.094913
...,...,...,...
86956,5.087669,0.011007,0.820213
86957,5.087353,0.010995,0.820213
86958,5.087036,0.010983,0.820213
86959,5.086720,0.010971,0.820213


In [7]:
XYZ_speeds[0]

,Velocity:0,Velocity:1,Velocity:2
0,1.851748,-4.954040,-2.546212
1,1.581876,-4.829920,-2.506701
2,1.429227,-4.875019,-2.531011
3,1.211334,-4.953471,-2.559960
4,0.926353,-4.790041,-2.531238
...,...,...,...
86956,2.347878,0.532362,0.062457
86957,2.414180,0.524881,0.066197
86958,2.479126,0.519707,0.069779
86959,2.542316,0.515082,0.073296


# Создание крыла

In [8]:
wing = wing1()

print(wing.n_parts)
print(wing.dims_zones)
print([len(wing.zones_inds[i]) for i in range(wing.n_parts)])

10
[(49, 17), (256, 49), (96, 17), (256, 48), (256, 48), (49, 16), (256, 49), (96, 16), (256, 48), (256, 48)]
[833, 12544, 1632, 12288, 12288, 784, 12544, 1536, 12288, 12288]


# Интегрирование по крылу

## Пробный прогон

In [17]:
ind = 0
xyz_coords = XYZ_coords[ind]
xyz_speeds = XYZ_speeds[ind]
cp, cf = Cp[ind], Cf[ind]
Sref = wing.S_REF
zones_inds = wing.zones_inds
dims_zones = wing.dims_zones

Cx_lab, Cy_lab, Cz_lab = sint.get_CxCyCz_LabSys_byPresFric(xyz_coords, xyz_speeds, cp, cf, 
                                                           Sref, zones_inds, dims_zones)

surface_CpCf_integrator: progress :  9/9  


In [18]:
Cx_lab, Cy_lab, Cz_lab

(0.03241120854311918, -0.38655941789834625, 0.041580846573889586)

In [19]:
# Переведмём эти величины в скоростную систему отсчёта
Cx_spd, Cy_spd, Cz_spd = sint.LabSys2SpdSys_CxCyCz(Cx_lab, Cy_lab, Cz_lab, AngAt=regimes['alpha'][ind])
Cx_spd, Cy_spd, Cz_spd

(0.05741477278833794, -0.38364334201286143, 0.041580846573889586)

## Подсчёт значений коэффициентов в лабораторной системе отсчёта для каждого обучающего примера

In [23]:
Nregimes = regimes.shape[0]
CxCyCz_lab__by_CpCf = np.zeros((Nregimes, 3), dtype=float)

In [24]:
Sref = wing.S_REF
zones_inds = wing.zones_inds
dims_zones = wing.dims_zones
for ind in range(Nregimes):
    print(ind, "/" , Nregimes)
    xyz_coords = XYZ_coords[ind]
    xyz_speeds = XYZ_speeds[ind]
    cp, cf = Cp[ind], Cf[ind]
    
    Cx_lab, Cy_lab, Cz_lab = sint.get_CxCyCz_LabSys_byPresFric(xyz_coords, 
                                                               xyz_speeds, 
                                                               cp, 
                                                               cf, 
                                                               Sref, 
                                                               zones_inds, 
                                                               dims_zones,
                                                               progress_bar=True)
    CxCyCz_lab__by_CpCf[ind] = np.array([Cx_lab, Cy_lab, Cz_lab])

0 / 64
surface_CpCf_integrator: progress :  9/9  
1 / 64
surface_CpCf_integrator: progress :  9/9  
2 / 64
surface_CpCf_integrator: progress :  9/9  
3 / 64
surface_CpCf_integrator: progress :  9/9  
4 / 64
surface_CpCf_integrator: progress :  9/9  
5 / 64
surface_CpCf_integrator: progress :  9/9  
6 / 64
surface_CpCf_integrator: progress :  9/9  
7 / 64
surface_CpCf_integrator: progress :  9/9  
8 / 64
surface_CpCf_integrator: progress :  9/9  
9 / 64
surface_CpCf_integrator: progress :  9/9  
10 / 64
surface_CpCf_integrator: progress :  9/9  
11 / 64
surface_CpCf_integrator: progress :  9/9  
12 / 64
surface_CpCf_integrator: progress :  9/9  
13 / 64
surface_CpCf_integrator: progress :  9/9  
14 / 64
surface_CpCf_integrator: progress :  9/9  
15 / 64
surface_CpCf_integrator: progress :  9/9  
16 / 64
surface_CpCf_integrator: progress :  9/9  
17 / 64
surface_CpCf_integrator: progress :  9/9  
18 / 64
surface_CpCf_integrator: progress :  9/9  
19 / 64
surface_CpCf_integrator: progress

In [25]:
table = pd.DataFrame(CxCyCz_lab__by_CpCf, columns=['Cx', 'Cy', 'Cz'])
table.to_csv('./my_CxCyCz_lab_by_CpCf', index=True)

# Извлечение оригинальных значений

In [27]:
CxCyCz_lab__by_CpCf

array([[ 0.02562994, -0.19044935,  0.02791825],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.   

In [22]:
coefCxyz_origin = pd.read_csv('./my csv/Pressure and Friction Conditioned Force Wing Coefficients', index_col=0)
coefCxyz_origin

,Cx,Cy,Cz
0,0.065854,-0.255098,0.020532
1,0.014904,-0.120466,0.023590
2,0.057367,0.655394,-0.037323
3,0.030255,0.601479,-0.033865
4,0.029740,0.439572,-0.025899
...,...,...,...
59,0.042182,0.618695,-0.036511
60,0.011811,0.268593,-0.012749
61,0.024298,-0.207613,0.030788
62,0.058127,0.602050,-0.041006
